## 预测房价：回归问题
预测一个连续值而不是离散的标签

In [1]:
from keras.datasets import boston_housing
(train_data,train_targets),(test_data,test_targets)=boston_housing.load_data()

ModuleNotFoundError: No module named 'keras'

In [10]:
print('训练集形状：',train_data.shape)
print('测试集形状：',test_data.shape)
print('有404个训练样本和102个测试样本，每个样本都有13个数值特征')
print("******************************")
print("前5个训练集目标：",train_targets[0:5])
print('目标是房屋价格的中位数')

训练集形状： (404, 13)
测试集形状： (102, 13)
有404个训练样本和102个测试样本，每个样本都有13个数值特征
******************************
前5个训练集目标： [15.2 42.3 50.  21.1 17.7]
目标是房屋价格的中位数


### 准备数据

In [16]:
# 特征标准化：即对于输入数据的每个特征（输入数据矩阵中的列），减去特征平均值，再除以标准差，这样得到的特征平均值为0，标准差为1
mean=train_data.mean(axis=0)
train_data-=mean
std=train_data.std(axis=0)
train_data/=std

test_data-=mean
test_data/=std

print('注意：用于测试数据标准化的均值和标准差都是在训练数据上计算得到的。在工作流程中，你不能使用在测试数据上计算得到的任何结果，即使是像数据标准化这么简单的事情也不行')

注意：用于测试数据标准化的均值和标准差都是在训练数据上计算得到的。在工作流程中，你不能使用在测试数据上计算得到的任何结果，即使是像数据标准化这么简单的事情也不行


### 构建网络

In [18]:
# 模型定义
from keras import models
from keras import layers
def build_model():
    model=models.Sequential()
    model.add(layers.Dense(64,activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(1))# 最后一层只有一个单元，没有激活，是一个线性层。
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
    return model

print('MSE:mean squared error  均方误差：预测值与目标值之差的平方')
print('MAE:mean absolute erroe 平均绝对误差：预测值与目标值之差的绝对值')

MSE:mean squared error  均方误差：预测值与目标值之差的平方
MAE:mean absolute erroe 平均绝对误差：预测值与目标值之差的绝对值


### 利用K折验证来验证你的方法

In [22]:
import numpy as np
k=4
num_val_samples=len(train_data)//k
num_epochs=100
all_scores=[]

for i in range(k):
    print('processing fold #',i)
    val_data=train_data[i* num_val_samples:(i+1)* num_val_samples] # 准备验证数据：第k个分区的数据
    val_targets=train_targets[i* num_val_samples:(i+1)* num_val_samples]
    
    # 准备训练数据：其他所有分区的数据  
    partial_train_data=np.concatenate([train_data[:i* num_val_samples],train_data[(i+1)* num_val_samples:]],axis=0)
    
    partial_train_targets=np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)
   
    model = build_model()#构建Keras模型（以编译）
    
    model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=0)#训练模型
    
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)# 在验证数据上评估模型
   
    all_scores.append(val_mae)

processing fold # 0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
processing fold # 1
processing fold # 2
processing fold # 3


In [23]:
all_scores

[2.0826985186869553, 2.2790751221156356, 2.8637446885061735, 2.390961466449322]

In [24]:
np.mean(all_scores)

2.4041199489395217